In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

## Step 1: Reading and Understanding the Data

In [ ]:
# read the data from csv file and load it into data frame called data
data = pd.read_csv('./day.csv')

# see the data
data.head()

In [ ]:
# get rows and columns
data.shape

In [ ]:
# as you can see from above data 'instant' column is just like an index
# so removing the 'instant' column as it will not contribute anythin to model
data = data.drop(columns=['instant'])
data.head()

In [ ]:
# Since we have the month and the Year in two seperate columns, we do not need the date column anymore, thus dropping it
data.drop('dteday', inplace=True, axis = 1)

Since the Model is to be built for <b>cnt</b> column, <b>casual</b> and <b>registed</b> are redundant here. It should not be used to build the model, thus dropping these two columns before further processing.

In [ ]:
# as 'cnt' varibale is combination of causal and registerd and we are predicting the 'cnt'
# so columns 'causal' and ''registerd' can be dropped 'cnt' is target variable
data = data.drop(columns=['casual', 'registered'])
data.head()

In [ ]:
# checking the data types of each column
data.info()

In [ ]:
# converting the actual caterhorical variables from numerical to categorical
cat_vars = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']
data[cat_vars] = data[cat_vars].astype(object)

In [ ]:
# checking the data types of each column
data.info()

As from the above info we can see all columns doesnot have any missing values and all are non-null. so null value checking is not required in this case

In [ ]:
# Converting the seasons variable values into their actual string values 
data['season'] = data['season'].replace([1, 2, 3, 4],['spring', 'summer', 'fall', 'winter'])
data.head()

In [ ]:
# Converting the weather variable values into their actual string values
data['weathersit'] = data['weathersit'].replace([1, 2, 3, 4],['Clear', 'Misty+Cloudy', 'Light Snow/Rain', 'Heavy Snow/Rain'])
data.head()

In [ ]:
data.describe()

## Step 2: Visualising the Data

#### Visualising Numeric Variables

Plot a pairplot of all the numeric variables

In [ ]:
# Checking linear relationship between the cnt variable and other numeric variables
num_vars = ['temp', 'atemp', 'hum', 'windspeed']
x = sns.pairplot(data, x_vars=num_vars, y_vars=['cnt'])
plt.show()

From the above plot we can see that cnt is having linear relationship with temp and atemp, where as humidity and windspeed is not having much linear and the data is spread across

#### Visualising Categorical Variables


In [ ]:
cat_vars

In [ ]:
# plot season vs cnt variable 
sns.boxplot(x='season', y='cnt', data=data)
plt.xlabel('Season')
plt.show()

So from the above plot it is observed that for summer and fall season the Count is more

In [ ]:
# plot Year vs cnt variable 
sns.boxplot(x='yr', y='cnt', data=data)
plt.xlabel('Year')
plt.show()

So from the above plot it is observed that for year 2019 Count is more

In [ ]:
# plot Month vs cnt variable 
sns.boxplot(x='mnth', y='cnt', data=data)
plt.xlabel('Month')
plt.show()

So from the above plot it is observed that there is demand is increasing from March to September. So these months count will be more as they falls under summer and fall season. 

In [ ]:
# plot holiday vs cnt variable 
sns.boxplot(x='holiday', y='cnt', data=data)
plt.xlabel('Holiday')
plt.show()

So from the above plot it is observed that holiday is not making any differnce in count

In [ ]:
# plot weekday vs cnt variable 
sns.boxplot(x='weekday', y='cnt', data=data)
plt.xlabel('weekday')
plt.show()

So from the above plot it is observed that Thursday and Friday is count is slightly more compare to other days

In [ ]:
# plot workingday vs cnt variable 
sns.boxplot(x='workingday', y='cnt', data=data)
plt.xlabel('workingday')
plt.show()

So from the above plot it is observed that workingday or not is not making any differnce in count

In [ ]:
# plot weathersit vs cnt variable 
sns.boxplot(x='weathersit', y='cnt', data=data)
plt.xlabel('weathersit')
plt.show()

So from the above plot it is observed that for clear weathersit day count is more

## Step 3: Data Preparation

In [ ]:
# create dummpy variables for categorical variables
dummy = pd.get_dummies(data[cat_vars], drop_first=True)
dummy.head()

In [ ]:
# now merge this dummy varibales to actual data frame and remove the categorical variables
data = pd.concat([data, dummy], axis=1)   #Axis=1 is for horizontal stacking
data = data.drop(columns=cat_vars, axis=1)
data.head()

In [ ]:
# get the number of rows and columns
data.shape

In [ ]:
data.head()

## Step 4: Splitting the Data into Training and Testing Sets

In [ ]:
np.random.seed(0)
# split data int0 70:30 
df_train, df_test = train_test_split(data, train_size=0.7, test_size = 0.3, random_state=100)

In [ ]:
print('Training data shape: ' , df_train.shape)
print('test data shape: ' , df_test.shape)

In [ ]:
# Checking the Train Data
df_train.head()

In [ ]:
# Checking the Test Data
df_test.head()

### Rescaling the Features 

In [ ]:
# apply min max scaling on the data

#Instantiating scalar object
scaler = MinMaxScaler()

# get the columns of traing data frame
cols = df_train.columns

# fit and transform the data
df_train[cols] = scaler.fit_transform(df_train[cols])

In [ ]:
df_train.describe()

In [ ]:
df_train.head()

In [ ]:
# Let's check the correlation coefficients to see which variables are highly correlated

plt.figure(figsize = (19,10))
sns.heatmap(df_train.corr(), annot = True, cmap="YlGnBu")
plt.show()

From the above plot temp and atemp is highly correlated to cnt compare to other variables.
Lets plot pair plot of them with cnt

In [ ]:
# plot temp vs cnt
plt.figure(figsize=[6,6])
plt.scatter(df_train.temp, df_train.cnt)
plt.show()

In [ ]:
# plot atemp vs cnt
plt.figure(figsize=[6,6])
plt.scatter(df_train.atemp, df_train.cnt)
plt.show()

From the above two plots temp and atemp having veru good linear relation ship with cnt variable.

## Step 5: Building a linear model
Since the number of columns is 29, which is manageable, we first build a model with all the columns, and then keep removing the columns based upon Statistical Significance and Co-Linearity. <br>
Here I am using Stats model to achieve this

In [ ]:
# separate x and y variables
y_train = df_train.pop('cnt')
X_train = df_train

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit 
lr_model1 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model1.summary()

The R-squared is a significant 85%, but there are insignificant variables and variables with strong multicollinearity. We need to get rid of them, in the following cells, we will follow the same process in an itrative manner till we build a robust model. First we will remove all columns with High P Values and then when the P Values are acceptable for all the columns, we will check their VIF and remove them.

In [ ]:
# Removing 'weekday_4' which is having high P-Value of 0.869 as it is far greater than parameter significance level of 5%
X_train = X_train.drop('weekday_4', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit 
lr_model2 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model2.summary()

In [ ]:
# Removing 'weekday_3' which is having high P-Value as it is greater than parameter significance level of 5%
X_train = X_train.drop('weekday_3', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit 
lr_model3 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model3.summary()


In [ ]:
# Removing 'atemp' which is having high P-Value as it is greater than parameter significance level of 5%
X_train = X_train.drop('atemp', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit 
lr_model4 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model4.summary()

In [ ]:
# Removing 'weekday_5' which is having high P-Value as it is greater than parameter significance level of 5%
X_train = X_train.drop('weekday_5', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit 
lr_model5 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model5.summary()

In [ ]:
# Removing 'mnth_7' which is having high P-Value as it is greater than parameter significance level of 5%
X_train = X_train.drop('mnth_7', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object
lr = sm.OLS(y_train, X_train_sm)

# instantiate the model object and fit 
lr_model6 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model6.summary()

In [ ]:
# Removing 'mnth_11' which is having high P-Value as it is greater than parameter significance level of 5%
X_train = X_train.drop('mnth_11', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit 
lr_model7 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model7.summary()

In [ ]:
# Removing 'mnth_12' which is having high P-Value as it is greater than parameter significance level of 5%
X_train = X_train.drop('mnth_12', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit 
lr_model8 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model8.summary()

In [ ]:
# Removing 'weekday_2' which is due to high P-Value
X_train = X_train.drop('weekday_2', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit 
lr_model9 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model9.summary()

In [ ]:
# Removing 'weekday_1' which is due to high P-Value
X_train = X_train.drop('weekday_1', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit
lr_model10 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model10.summary()

In [ ]:
# Removing 'mnth_2' which due to high P-Value
X_train = X_train.drop('mnth_2', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit
lr_model11 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model11.summary()

In [ ]:
# Removing 'season_summer' which due to high P-Value
X_train = X_train.drop('season_summer', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit
lr_model12 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model12.summary()

In [ ]:
# Removing 'holiday_1' which due to high P-Value
X_train = X_train.drop('holiday_1', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit
lr_model13 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model13.summary()

From the above model we can see that all the parameters p-values is less than the parameter siginificance level of 5%
Now look into VIF of those parameters and remove any params is having more VIF

In [ ]:
# Checking VIF (Variance Inflation Factor - MultiColinearity)
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

Humidity and Temperature have a high VIF, which means they have multicolinearity and one of them must be removed and checked again.

In [ ]:
# Removing 'hum' due to high VIF
X_train = X_train.drop('hum', axis=1)

# add constant
X_train_sm = sm.add_constant(X_train)

# instantiate the model object and fit
lr_model14 = sm.OLS(y_train, X_train_sm).fit()

# get the summary of the model
lr_model14.summary()

In [ ]:
#Checking the VIF Again
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

This looks like an acceptable model. We keep the <b>temp</b> variable, because from our EDA, we have seen that Temperature has a direct colinearity with the booking count. On colder days, the bookings are less, whereas on hotter, summer time, the bookings are up significantly. Thus as per business understanding, we finalize this model as the final model.

In [ ]:
# Checking the co-efficients of the final model lr_model14
print(lr_model14.summary())

## Step 7: Residual Analysis of the train data

So, now to check if the error terms are also normally distributed (which is infact, one of the major assumptions of linear regression), let us plot the histogram of the error terms and see what it looks like.

In [ ]:
y_train_pred = lr_model14.predict(X_train_sm)

In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_pred), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label


From the above plot t has been seen that residuals of training data is following normal distribution. Now make prediction on the test data

# Validating the assumptions of Linear Regression
1. Linear Relationship
2. No Multicollinearity
3. Independence of residuals (absence of auto-correlation)
4. Homoscedasticity- 
5. Normality of Errors

##  1. Linear Relationship
One of the most important assumptions is that a linear relationship is said to exist between the dependent and the 
independent variables. If you try to fit a linear relationship in a non-linear data set, the proposed algorithm won’t 
capture the trend as a linear graph, resulting in an inefficient model. Thus, it would result in inaccurate predictions.

##### How can you determine if the assumption is met?

The simple way to determine if this assumption is met or not is by creating a scatter plot x vs y. If the data points fall on a straight line in the graph, there is a linear relationship between the dependent and the independent variables, and the assumption holds.

In [ ]:
# Validating Linear Relationship
sm.graphics.plot_ccpr(lr_model14, 'temp')
plt.show()

The partial residual plot represents the relationship between the predictor and the dependent variable while taking into account all the other variables. As we can see in the above graph, the linearity is well respected.

## 2. No Multicollinearity

The independent variables shouldn’t be correlated. If multicollinearity exists between the independent variables, 
it is challenging to predict the outcome of the model. In essence, it is difficult to explain the relationship between 
the dependent and the independent variables. In other words, it is unclear which independent variables explain the 
dependent variable.

The standard errors tend to inflate with correlated variables, thus widening the confidence intervals leading to 
imprecise estimates.

##### How to determine if the assumption is met?

Use a scatter plot to visualise the correlation between the variables. Another way is to determine the VIF
(Variance Inflation Factor). VIF<=4 implies no multicollinearity, whereas VIF>=10 implies serious multicollinearity.


In [ ]:
# Validating Multi Colinearity
plt.figure(figsize=(15,8))
sns.heatmap(X_train.corr(), annot=True, cmap='YlGnBu')
plt.show()

All variables have less than 0.56 correlation with eachother. Checking the VIF now.

In [ ]:
print(vif)

Taking 10 as the maximum VIF permissible for this model, we decide on keeping these colmns based upon business assumptions.

## 3. No auto-correlation or independence
The residuals (error terms) are independent of each other. In other words, there is no correlation between the 
consecutive error terms of the time series data. The presence of correlation in the error terms drastically 
reduces the accuracy of the model. If the error terms are correlated, the estimated standard error tries to 
deflate the true standard error.

###### How to determine if the assumption is met?

Conduct a Durbin-Watson (DW) statistic test. The values should fall between 0-4. If DW=2, no auto-correlation; 
if DW lies between 0 and 2, it means that there exists a positive correlation. If DW lies between 2 and 4, 
it means there is a negative correlation. Another method is to plot a graph against residuals vs time and see 
patterns in residual values.

In [ ]:
# Independence of residuals (absence of auto-correlation)
# Autocorrelation refers to the fact that observations’ errors are correlated
# To verify that the observations are not auto-correlated, we can use the Durbin-Watson test. 
# The test will output values between 0 and 4. The closer it is to 2, the less auto-correlation there is between the various variables
# (0–2: positive auto-correlation, 2–4: negative auto-correlation)

print('The Durbin-Watson value for Model No.19 is',round(sm.stats.stattools.durbin_watson((y_train - y_train_pred)),4))

There is almost nill auto-correlation

## 4. Homoscedasticity
Homoscedasticity means the residuals have constant variance at every level of x. The absence of this phenomenon is 
known as heteroscedasticity. Heteroscedasticity generally arises in the presence of outliers and extreme values.

##### How to determine if the assumption is met?
Create a scatter plot that shows residual vs fitted value. If the data points are spread across equally without a 
prominent pattern, it means the residuals have constant variance (homoscedasticity). Otherwise, if a funnel-shaped 
pattern is seen, it means the residuals are not distributed equally and depicts a non-constant variance (heteroscedasticity).

In [ ]:
# Validating Homoscedasticity : The residuals have constant variance with respect to the dependent variable
y_train_pred = lr_model14.predict(X_train_sm)
sns.scatterplot(y_train,(y_train - y_train_pred))
plt.plot(y_train,(y_train - y_train), '-r')
plt.xlabel('Count')
plt.ylabel('Residual')
plt.show()

As we can see in the above plot, Homoscedasticity is well respected since the variance of the residuals are almost constant.

## 5. Normal distribution of error terms
The last assumption that needs to be checked for linear regression is the error terms’ normal distribution. 
If the error terms don’t follow a normal distribution, confidence intervals may become too wide or narrow.

##### How to determine if the assumption is met?

Check the assumption using a Q-Q (Quantile-Quantile) plot. If the data points on the graph form a straight diagonal line, 
the assumption is met.
You can also check for the error terms’ normality using statistical tests like the Kolmogorov-Smironov or Shapiro-Wilk test.

In [ ]:
# Normality of Errors
y_train_pred = lr_model14.predict(X_train_sm)

# Ploting the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_pred))
fig.suptitle('Error Terms')                  
plt.xlabel('Errors')     
plt.show()

In [ ]:
sm.qqplot((y_train - y_train_pred), fit=True, line='45')
plt.show()

The error terms are normally distributed

## Step 8: Making Predictions Using the Final Model


In [ ]:
# Scaling the Test Dataset with the Scaler of the Training Set
cols = df_test.columns
df_test[cols] = scaler.transform(df_test[cols])

In [ ]:
# see the transformed data
df_test.describe()

In [ ]:
# Dividing data into X_test and y_test
y_test = df_test.pop('cnt')
X_test = df_test

In [ ]:
X_test.head()

In [ ]:
# as in the training data some of the non-significant columns is removed for model building.
# so for test data also we have to remove those columns or filter data only for significant columns
# get the training data final columns
final_columns = X_train.columns

In [ ]:
# print the data
X_test[final_columns].head()

In [ ]:
# sub set the test data based on significant columns which is acquired from training data
X_test_m14 = X_test[final_columns]
# Adding the constant column
X_test_m14 = sm.add_constant(X_test_m14)

In [ ]:
# Making prediction using Model 14
y_test_pred = lr_model14.predict(X_test_m14)

## Step 9: Model Evaluation

Let's now plot the graph for actual versus predicted values of test data

In [ ]:
# Plotting y_test and y_pred to understand the spread
fig = plt.figure()
plt.scatter(y_test, y_test_pred)
fig.suptitle('y_test vs y_pred', fontsize = 20)              # Plot heading 
plt.xlabel('y_test', fontsize = 18)                          # X-label
plt.ylabel('y_test_pred', fontsize = 16)  

From the above plot we can see actual and predicted are aligned each other

In [ ]:
# Now lets check the r2 and adj-r2 for training and testing data
r2_train = r2_score(y_true=y_train, y_pred=y_train_pred)
r2_test = r2_score(y_true=y_test, y_pred=y_test_pred)
print("Training data r2-value : ", r2_train)
print("Testing data r2-value : ", r2_test)

So as we can see above r2 value for training and testing is near to each other. So we can tell that model is performing good.

The equation of the best fitted line developed by Model 14 is:

$ 𝑐𝑛𝑡=0.1693+(0.4121*temp-0.1544*windspeed-0.0759*season_spring+0.0833*season_winter+0.2352*yr_1+0.0620*mnth_3+0.0705*mnth_4+0.0880*mnth_5+0.0633*mnth_6+0.0512*mnth_8+0.1124*mnth_9+0.0500*mnth_10+0.0651*weekday_6+0.0533*workingday_1-0.2968*weathersit_Light Snow/Rain-0.0838*weathersit_Misty+Cloudy) $


Since the bookings increase on Clear weather days with hotter temperature, the company must increase their bike availibilty and promotions during the summer months to further increase their booking count.

An R-Squared value of 0.82 on the test data signifies that the model is a very good predictor and 82% of the variance is captured by the model.

# Building a linear model using RFE(Recursive feature elimination) and sklearn

As in the above steps i have build the model using manually removing the insignificat varaibles one by one at a time.
Now in below I am building the model using automatic way of removing the insigificant parameters using RFE Method
and build the model using that and see both are giving same kind of accuracy or not

In [ ]:
#Dividing into X and Y sets for the model building
y_train = y_train
X_train = df_train

In [ ]:
X_train.head()

In [ ]:
# as from the manual mode finally we got 16parameters as significant so I am also using same count for RFE model
# Running RFE with the output number of the variable equal to 16
# create linear regression model object and fit train and test data
lm = LinearRegression()
lm.fit(X_train, y_train)

# create rfe object
rfe = RFE(lm, 16)  
# fit the model
rfe = rfe.fit(X_train, y_train)

In [ ]:
# print the parameter ranks
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
# get the only significant params
col = X_train.columns[rfe.support_]
col

In [ ]:
# print insignificant params
X_train.columns[~rfe.support_]

### Building model using statsmodel, for the detailed statistics

In [ ]:
# Creating X_test dataframe with RFE selected variables
X_train_rfe = X_train[col]

In [ ]:
# Adding a constant variable 
import statsmodels.api as sm  
X_train_rfe = sm.add_constant(X_train_rfe)

In [ ]:
# Running the linear model
lm = sm.OLS(y_train, X_train_rfe).fit()   

In [ ]:
#Let's see the summary of our linear model
print(lm.summary())

In [ ]:
# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_rfe
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

From above VIF table everything is under 5 so all the parameters are siginificant

## Residual Analysis of the train data

So, now to check if the error terms are also normally distributed (which is infact, one of the major assumptions of linear regression), let us plot the histogram of the error terms and see what it looks like.

In [ ]:
y_train_pred = lm.predict(X_train_rfe)

In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_pred), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label

From the above plot its clear that resisuals is following Normal Distribution. So model is good.

# Validating the assumptions of Linear Regression
1. Linear Relationship
2. No Multicollinearity
3. Independence of residuals (absence of auto-correlation)
4. Homoscedasticity- 
5. Normality of Errors

##  1. Linear Relationship
One of the most important assumptions is that a linear relationship is said to exist between the dependent and the 
independent variables. If you try to fit a linear relationship in a non-linear data set, the proposed algorithm won’t 
capture the trend as a linear graph, resulting in an inefficient model. Thus, it would result in inaccurate predictions.

##### How can you determine if the assumption is met?

The simple way to determine if this assumption is met or not is by creating a scatter plot x vs y. If the data points fall on a straight line in the graph, there is a linear relationship between the dependent and the independent variables, and the assumption holds.

In [ ]:
# Validating Linear Relationship
sm.graphics.plot_ccpr(lr_model14, 'temp')
plt.show()

The partial residual plot represents the relationship between the predictor and the dependent variable while taking into account all the other variables. As we can see in the above graph, the linearity is well respected.

## 2. No Multicollinearity

The independent variables shouldn’t be correlated. If multicollinearity exists between the independent variables, 
it is challenging to predict the outcome of the model. In essence, it is difficult to explain the relationship between 
the dependent and the independent variables. In other words, it is unclear which independent variables explain the 
dependent variable.

The standard errors tend to inflate with correlated variables, thus widening the confidence intervals leading to 
imprecise estimates.

##### How to determine if the assumption is met?

Use a scatter plot to visualise the correlation between the variables. Another way is to determine the VIF
(Variance Inflation Factor). VIF<=4 implies no multicollinearity, whereas VIF>=10 implies serious multicollinearity.


In [ ]:
# Validating Multi Colinearity
plt.figure(figsize=(15,8))
sns.heatmap(X_train.corr(), annot=True, cmap='YlGnBu')
plt.show()

All variables have less than 0.56 correlation with eachother. Checking the VIF now.

In [ ]:
print(vif)

Taking 10 as the maximum VIF permissible for this model, we decide on keeping these colmns based upon business assumptions.

## 3. No auto-correlation or independence
The residuals (error terms) are independent of each other. In other words, there is no correlation between the 
consecutive error terms of the time series data. The presence of correlation in the error terms drastically 
reduces the accuracy of the model. If the error terms are correlated, the estimated standard error tries to 
deflate the true standard error.

###### How to determine if the assumption is met?

Conduct a Durbin-Watson (DW) statistic test. The values should fall between 0-4. If DW=2, no auto-correlation; 
if DW lies between 0 and 2, it means that there exists a positive correlation. If DW lies between 2 and 4, 
it means there is a negative correlation. Another method is to plot a graph against residuals vs time and see 
patterns in residual values.

In [ ]:
# Independence of residuals (absence of auto-correlation)
# Autocorrelation refers to the fact that observations’ errors are correlated
# To verify that the observations are not auto-correlated, we can use the Durbin-Watson test. 
# The test will output values between 0 and 4. The closer it is to 2, the less auto-correlation there is between the various variables
# (0–2: positive auto-correlation, 2–4: negative auto-correlation)

print('The Durbin-Watson value for Model No.19 is',round(sm.stats.stattools.durbin_watson((y_train - y_train_pred)),4))

There is almost nill auto-correlation

## 4. Homoscedasticity
Homoscedasticity means the residuals have constant variance at every level of x. The absence of this phenomenon is 
known as heteroscedasticity. Heteroscedasticity generally arises in the presence of outliers and extreme values.

##### How to determine if the assumption is met?
Create a scatter plot that shows residual vs fitted value. If the data points are spread across equally without a 
prominent pattern, it means the residuals have constant variance (homoscedasticity). Otherwise, if a funnel-shaped 
pattern is seen, it means the residuals are not distributed equally and depicts a non-constant variance (heteroscedasticity).

In [ ]:
# Validating Homoscedasticity : The residuals have constant variance with respect to the dependent variable
y_train_pred = lr_model14.predict(X_train_sm)
sns.scatterplot(y_train,(y_train - y_train_pred))
plt.plot(y_train,(y_train - y_train), '-r')
plt.xlabel('Count')
plt.ylabel('Residual')
plt.show()

As we can see in the above plot, Homoscedasticity is well respected since the variance of the residuals are almost constant.

## 5. Normal distribution of error terms
The last assumption that needs to be checked for linear regression is the error terms’ normal distribution. 
If the error terms don’t follow a normal distribution, confidence intervals may become too wide or narrow.

##### How to determine if the assumption is met?

Check the assumption using a Q-Q (Quantile-Quantile) plot. If the data points on the graph form a straight diagonal line, 
the assumption is met.
You can also check for the error terms’ normality using statistical tests like the Kolmogorov-Smironov or Shapiro-Wilk test.

In [ ]:
# Normality of Errors
y_train_pred = lr_model14.predict(X_train_sm)

# Ploting the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_pred))
fig.suptitle('Error Terms')                  
plt.xlabel('Errors')     
plt.show()

In [ ]:
sm.qqplot((y_train - y_train_pred), fit=True, line='45')
plt.show()

The error terms are normally distributed

## Making Predictions Using the Final Model

In [ ]:
# see the transformed data
df_test.describe()

In [ ]:
# Dividing data into X_test and y_test
y_test = y_test
X_test = df_test

In [ ]:
X_test.head()

In [ ]:
# as in the training data some of the non-significant columns is removed for model building.
# so for test data also we have to remove those columns or filter data only for significant columns
# get the training data final columns
final_columns = list(X_train_rfe.columns)
final_columns.remove("const")

In [ ]:
# print the data
X_train_rfe[final_columns].head()

In [ ]:
# sub set the test data based on significant columns which is acquired from training data
X_test_rfe = X_test[final_columns]
# Adding the constant column
X_test_rfe = sm.add_constant(X_test_rfe)

In [ ]:
# Making prediction using Model
y_test_pred = lm.predict(X_test_rfe)

## Model Evaluation

Let's now plot the graph for actual versus predicted values of test data

In [ ]:
# Plotting y_test and y_pred to understand the spread
fig = plt.figure()
plt.scatter(y_test, y_test_pred)
fig.suptitle('y_test vs y_pred', fontsize = 20)              # Plot heading 
plt.xlabel('y_test', fontsize = 18)                          # X-label
plt.ylabel('y_test_pred', fontsize = 16)  

From the above plot we can see actual and predicted are aligned each other

In [ ]:
# Now lets check the r2 and adj-r2 for training and testing data
r2_train = r2_score(y_true=y_train, y_pred=y_train_pred)
r2_test = r2_score(y_true=y_test, y_pred=y_test_pred)
print("Training data r2-value : ", r2_train)
print("Testing data r2-value : ", r2_test)

So as we can see above r2 value for training and testing is near to each other. So we can tell that model is performing good.

The equation of the best fitted line developed by Model 14 is:

$ 𝑐𝑛𝑡=0.3031+(0.4385*temp-0.1620*hum-0.1823*windspeed-0.0699*season_spring+0.0949*season_winter+0.2313*yr_1+0.0628*mnth_3+0.0703*mnth_4+0.0983*mnth_5+0.0586*mnth_6+0.0548*mnth_8+0.1221*mnth_9+0.0488*mnth_10-0.0859*holiday_1-0.2481*weathersit_Light Snow/Rain-0.0567*weathersit_Misty+Cloudy) $


## Final Conclusion

From the above two models we build one using manual elimination of insginificant parameters and other using RFE method of
eliminating insginificant parameters it is observed that both models is having same accuracy of r2 values for traing and test data.

Since the bookings increase on Clear weather days with hotter temperature, the company must increase their bike availibilty and promotions during the summer months to further increase their booking count.

An R-Squared value of 0.82 on the test data signifies that the model is a very good predictor and 82% of the variance is captured by the model.

# Validating the assumptions of Linear Regression
1. Linear Relationship
2. No Multicollinearity
3. Independence of residuals (absence of auto-correlation)
4. Homoscedasticity- 
5. Normality of Errors

##  1. Linear Relationship
One of the most important assumptions is that a linear relationship is said to exist between the dependent and the 
independent variables. If you try to fit a linear relationship in a non-linear data set, the proposed algorithm won’t 
capture the trend as a linear graph, resulting in an inefficient model. Thus, it would result in inaccurate predictions.

##### How can you determine if the assumption is met?

The simple way to determine if this assumption is met or not is by creating a scatter plot x vs y. If the data points fall on a straight line in the graph, there is a linear relationship between the dependent and the independent variables, and the assumption holds.

In [ ]:
# Validating Linear Relationship
sm.graphics.plot_ccpr(lr_model14, 'temp')
plt.show()

The partial residual plot represents the relationship between the predictor and the dependent variable while taking into account all the other variables. As we can see in the above graph, the linearity is well respected.

## 2. No Multicollinearity

The independent variables shouldn’t be correlated. If multicollinearity exists between the independent variables, 
it is challenging to predict the outcome of the model. In essence, it is difficult to explain the relationship between 
the dependent and the independent variables. In other words, it is unclear which independent variables explain the 
dependent variable.

The standard errors tend to inflate with correlated variables, thus widening the confidence intervals leading to 
imprecise estimates.

##### How to determine if the assumption is met?

Use a scatter plot to visualise the correlation between the variables. Another way is to determine the VIF
(Variance Inflation Factor). VIF<=4 implies no multicollinearity, whereas VIF>=10 implies serious multicollinearity.


In [ ]:
# Validating Multi Colinearity
plt.figure(figsize=(15,8))
sns.heatmap(X_train.corr(), annot=True, cmap='YlGnBu')
plt.show()

All variables have less than 0.56 correlation with eachother. Checking the VIF now.

In [ ]:
print(vif)

Taking 10 as the maximum VIF permissible for this model, we decide on keeping these colmns based upon business assumptions.

## 3. No auto-correlation or independence
The residuals (error terms) are independent of each other. In other words, there is no correlation between the 
consecutive error terms of the time series data. The presence of correlation in the error terms drastically 
reduces the accuracy of the model. If the error terms are correlated, the estimated standard error tries to 
deflate the true standard error.

###### How to determine if the assumption is met?

Conduct a Durbin-Watson (DW) statistic test. The values should fall between 0-4. If DW=2, no auto-correlation; 
if DW lies between 0 and 2, it means that there exists a positive correlation. If DW lies between 2 and 4, 
it means there is a negative correlation. Another method is to plot a graph against residuals vs time and see 
patterns in residual values.

In [ ]:
# Independence of residuals (absence of auto-correlation)
# Autocorrelation refers to the fact that observations’ errors are correlated
# To verify that the observations are not auto-correlated, we can use the Durbin-Watson test. 
# The test will output values between 0 and 4. The closer it is to 2, the less auto-correlation there is between the various variables
# (0–2: positive auto-correlation, 2–4: negative auto-correlation)

print('The Durbin-Watson value for Model No.19 is',round(sm.stats.stattools.durbin_watson((y_train - y_train_pred)),4))

There is almost nill auto-correlation

## 4. Homoscedasticity
Homoscedasticity means the residuals have constant variance at every level of x. The absence of this phenomenon is 
known as heteroscedasticity. Heteroscedasticity generally arises in the presence of outliers and extreme values.

##### How to determine if the assumption is met?
Create a scatter plot that shows residual vs fitted value. If the data points are spread across equally without a 
prominent pattern, it means the residuals have constant variance (homoscedasticity). Otherwise, if a funnel-shaped 
pattern is seen, it means the residuals are not distributed equally and depicts a non-constant variance (heteroscedasticity).

In [ ]:
# Validating Homoscedasticity : The residuals have constant variance with respect to the dependent variable
y_train_pred = lr_model14.predict(X_train_sm)
sns.scatterplot(y_train,(y_train - y_train_pred))
plt.plot(y_train,(y_train - y_train), '-r')
plt.xlabel('Count')
plt.ylabel('Residual')
plt.show()

As we can see in the above plot, Homoscedasticity is well respected since the variance of the residuals are almost constant.

## 5. Normal distribution of error terms
The last assumption that needs to be checked for linear regression is the error terms’ normal distribution. 
If the error terms don’t follow a normal distribution, confidence intervals may become too wide or narrow.

##### How to determine if the assumption is met?

Check the assumption using a Q-Q (Quantile-Quantile) plot. If the data points on the graph form a straight diagonal line, 
the assumption is met.
You can also check for the error terms’ normality using statistical tests like the Kolmogorov-Smironov or Shapiro-Wilk test.

In [ ]:
# Normality of Errors
y_train_pred = lr_model14.predict(X_train_sm)

# Ploting the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_pred))
fig.suptitle('Error Terms')                  
plt.xlabel('Errors')     
plt.show()

In [ ]:
sm.qqplot((y_train - y_train_pred), fit=True, line='45')
plt.show()

The error terms are normally distributed

# Validating the assumptions of Linear Regression
1. Linear Relationship
2. No Multicollinearity
3. Independence of residuals (absence of auto-correlation)
4. Homoscedasticity- 
5. Normality of Errors

##  1. Linear Relationship
One of the most important assumptions is that a linear relationship is said to exist between the dependent and the 
independent variables. If you try to fit a linear relationship in a non-linear data set, the proposed algorithm won’t 
capture the trend as a linear graph, resulting in an inefficient model. Thus, it would result in inaccurate predictions.

##### How can you determine if the assumption is met?

The simple way to determine if this assumption is met or not is by creating a scatter plot x vs y. If the data points fall on a straight line in the graph, there is a linear relationship between the dependent and the independent variables, and the assumption holds.

In [ ]:
# Validating Linear Relationship
sm.graphics.plot_ccpr(lr_model14, 'temp')
plt.show()

The partial residual plot represents the relationship between the predictor and the dependent variable while taking into account all the other variables. As we can see in the above graph, the linearity is well respected.

## 2. No Multicollinearity

The independent variables shouldn’t be correlated. If multicollinearity exists between the independent variables, 
it is challenging to predict the outcome of the model. In essence, it is difficult to explain the relationship between 
the dependent and the independent variables. In other words, it is unclear which independent variables explain the 
dependent variable.

The standard errors tend to inflate with correlated variables, thus widening the confidence intervals leading to 
imprecise estimates.

##### How to determine if the assumption is met?

Use a scatter plot to visualise the correlation between the variables. Another way is to determine the VIF
(Variance Inflation Factor). VIF<=4 implies no multicollinearity, whereas VIF>=10 implies serious multicollinearity.


In [ ]:
# Validating Multi Colinearity
plt.figure(figsize=(15,8))
sns.heatmap(X_train.corr(), annot=True, cmap='YlGnBu')
plt.show()

All variables have less than 0.56 correlation with eachother. Checking the VIF now.

In [ ]:
print(vif)

Taking 10 as the maximum VIF permissible for this model, we decide on keeping these colmns based upon business assumptions.

## 3. No auto-correlation or independence
The residuals (error terms) are independent of each other. In other words, there is no correlation between the 
consecutive error terms of the time series data. The presence of correlation in the error terms drastically 
reduces the accuracy of the model. If the error terms are correlated, the estimated standard error tries to 
deflate the true standard error.

###### How to determine if the assumption is met?

Conduct a Durbin-Watson (DW) statistic test. The values should fall between 0-4. If DW=2, no auto-correlation; 
if DW lies between 0 and 2, it means that there exists a positive correlation. If DW lies between 2 and 4, 
it means there is a negative correlation. Another method is to plot a graph against residuals vs time and see 
patterns in residual values.

In [ ]:
# Independence of residuals (absence of auto-correlation)
# Autocorrelation refers to the fact that observations’ errors are correlated
# To verify that the observations are not auto-correlated, we can use the Durbin-Watson test. 
# The test will output values between 0 and 4. The closer it is to 2, the less auto-correlation there is between the various variables
# (0–2: positive auto-correlation, 2–4: negative auto-correlation)

print('The Durbin-Watson value for Model No.19 is',round(sm.stats.stattools.durbin_watson((y_train - y_train_pred)),4))

There is almost nill auto-correlation

## 4. Homoscedasticity
Homoscedasticity means the residuals have constant variance at every level of x. The absence of this phenomenon is 
known as heteroscedasticity. Heteroscedasticity generally arises in the presence of outliers and extreme values.

##### How to determine if the assumption is met?
Create a scatter plot that shows residual vs fitted value. If the data points are spread across equally without a 
prominent pattern, it means the residuals have constant variance (homoscedasticity). Otherwise, if a funnel-shaped 
pattern is seen, it means the residuals are not distributed equally and depicts a non-constant variance (heteroscedasticity).

In [ ]:
# Validating Homoscedasticity : The residuals have constant variance with respect to the dependent variable
y_train_pred = lr_model14.predict(X_train_sm)
sns.scatterplot(y_train,(y_train - y_train_pred))
plt.plot(y_train,(y_train - y_train), '-r')
plt.xlabel('Count')
plt.ylabel('Residual')
plt.show()

As we can see in the above plot, Homoscedasticity is well respected since the variance of the residuals are almost constant.

## 5. Normal distribution of error terms
The last assumption that needs to be checked for linear regression is the error terms’ normal distribution. 
If the error terms don’t follow a normal distribution, confidence intervals may become too wide or narrow.

##### How to determine if the assumption is met?

Check the assumption using a Q-Q (Quantile-Quantile) plot. If the data points on the graph form a straight diagonal line, 
the assumption is met.
You can also check for the error terms’ normality using statistical tests like the Kolmogorov-Smironov or Shapiro-Wilk test.

In [ ]:
# Normality of Errors
y_train_pred = lr_model14.predict(X_train_sm)

# Ploting the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_pred))
fig.suptitle('Error Terms')                  
plt.xlabel('Errors')     
plt.show()

In [ ]:
sm.qqplot((y_train - y_train_pred), fit=True, line='45')
plt.show()

The error terms are normally distributed